In [ ]:
from abc import ABC, abstractmethod
import os
import json
import requests
from google.cloud import servicedirectory
from msal import ConfidentialClientApplication
from google.cloud import kms
from urllib.parse import urljoin

class MCPBaseClient(ABC):
    def __init__(self, context_type):
        self.context_type = context_type
        self.mcp_server_url = self._discover_mcp_server()
        self.manifest = self._fetch_manifest()
        self.service_url = self._extract_endpoint()
        self.auth_client = self._init_auth_client()

    def _discover_mcp_server(self):
        """Step 1: Discover MCP Server via Service Directory"""
        client = servicedirectory.LookupServiceClient()
        request = servicedirectory.ResolveServiceRequest(
            name=f"projects/{os.getenv('GCP_PROJECT')}/locations/global/namespaces/mcp/services/{self.context_type}"
        )
        response = client.resolve_service(request)
        return f"https://{response.service.endpoints[0].address}"

    def _fetch_manifest(self):
        """Step 2: Fetch manifest from MCP Server"""
        manifest_url = urljoin(self.mcp_server_url, "/.well-known/mcp-manifest")
        response = requests.get(manifest_url, timeout=5)
        response.raise_for_status()
        return response.json()

    def _extract_endpoint(self):
        """Step 3: Extract endpoint from manifest based on subclass type"""
        endpoint_type = self._get_endpoint_type()  # Implemented by subclasses
        return self.manifest["endpoints"][endpoint_type]

    def _init_auth_client(self):
        return ConfidentialClientApplication(
            client_id=os.getenv("AZURE_CLIENT_ID"),
            client_credential=os.getenv("AZURE_CLIENT_SECRET"),
            authority=f"https://login.microsoftonline.com/{os.getenv('AZURE_TENANT_ID')}"
        )

    def _get_token(self):
        """Get OAuth token using manifest's audience"""
        result = self.auth_client.acquire_token_for_client(
            scopes=[f"api://{self.manifest['audience']}/.default"],
            enable_pkce=True
        )
        return result["access_token"]

    # ABSTRACT METHODS
    @abstractmethod
    def _get_endpoint_type(self):
        """Return 'rest', 'graph', or 'jdbc'"""
        pass

    @abstractmethod
    def send_request(self, payload, token):
        pass

    def get_context(self, payload):
        token = self._get_token()
        response = self.send_request(payload, token)
        self._verify_signature(response)
        return response["context"]

    def _verify_signature(self, signed_payload):
        """Verify using KMS key from manifest"""
        kms_client = kms.KeyManagementServiceClient()
        public_key = kms_client.get_public_key(
            name=self.manifest["kms_key_path"]
        )
        verification = kms_client.asymmetric_verify(
            name=public_key.name,
            data=json.dumps(signed_payload["context"]).encode(),
            signature=bytes.fromhex(signed_payload["signature"])
        )
        if not verification.verified:
            raise SecurityError("Signature verification failed")